# American Airlines Random Forest Regression Model

## Import Data

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [4]:
features = pd.read_csv('all_tight_turns_seconds.csv')

## Establish a Baseline

If one expects that a plane in a tight turn situation will be turned over in a time less than or equal to MOGT, they will be correct 48.1% of the time.  Our accuracy rate needs to be significantly better than this in order to provide utility to the customer.

In [ ]:
print('Baseline: {:.2%}'.format(features.Under_MOGT.sum() / len(features.Under_MOGT)))

## Outlier Detection

In [ ]:
# Histogram
hist = features['TURN_TIME'].hist()

In [ ]:
# Scatter plot of available ground time vs turn time
plt.figure(figsize=(15,7))
plt.scatter(features['AVL_GRND_TM'],features['TURN_TIME'],s=100,marker='o',color='#b3b3ff')
plt.title('AVL_GRND_TM vs TURN_TIME',fontsize=20)  
plt.xlabel('AVL_GRND_TM',fontsize=15)  
plt.ylabel('TURN_TIME',fontsize=15)  
plt.show()

## Remove Outliers 
Outliers with a turn time greater than 8 hours will be analyzed separately. 

In [ ]:
features.shape

In [5]:
features.drop(features[features.TURN_TIME > 28800].index, inplace=True)
features.drop(features[features.AVL_GRND_TM < -200].index, inplace=True)

In [ ]:
features.shape

In [ ]:
# Scatter plot of available ground time vs turn time
plt.figure(figsize=(15,7))
plt.scatter(features['AVL_GRND_TM'],features['TURN_TIME'],s=100,marker='o',color='#b3b3ff')
plt.title('AVL_GRND_TM vs TURN_TIME',fontsize=20)  
plt.xlabel('AVL_GRND_TM',fontsize=15)  
plt.ylabel('TURN_TIME',fontsize=15)  
plt.show()

## Pre-process the Data

#### Dummy variables

Here we convert categorical columns into one-hot encoded features

In [6]:
features = pd.get_dummies(features)
features.columns

Index(['INB_FLIGHT', 'INB_ACT_DTTM', 'SEATS', 'INB_PAX', 'OUTB_PAX', 'MOGT',
       'AVL_GRND_TM', 'OPERAT_FLIGHT_NBR', 'RQRD_BOARD_MIN', 'SKD_DTTM',
       ...
       'PRIMARY_DELAY_REASON_MAINTENANCE',
       'PRIMARY_DELAY_REASON_MISCELLANEOUS', 'PRIMARY_DELAY_REASON_NO DELAY',
       'PRIMARY_DELAY_REASON_OPERATIONAL CODES',
       'PRIMARY_DELAY_REASON_PASSENGER SERVICE', 'PRIMARY_DELAY_REASON_PILOT',
       'PRIMARY_DELAY_REASON_SECURITY', 'PRIMARY_DELAY_REASON_SERVICING',
       'PRIMARY_DELAY_REASON_UNKNOWN', 'PRIMARY_DELAY_REASON_WEATHER'],
      dtype='object', length=2248)

In [ ]:
features.head()

#### Creating new columns

The following new columns are created below:
* inb_pct_full - the percentage of seats occupied on the inbound flight
* outb_pct_full - the percentage of seats occupied on the outbound flight
* sched_month - the month the flight was scheduled to leave
* sched_hour - the hour the flight was scheduled to leave
* arrival_hour - the hour the inbound flight arrived

In [7]:
features['inb_pct_full'] = features['INB_PAX'] / features['SEATS']
features['outb_pct_full'] = features['OUTB_PAX'] / features['SEATS']
features['sched_month'] = pd.to_datetime(features['SKD_DTTM']).dt.month
features['sched_day_of_week'] = pd.to_datetime(features['SKD_DTTM']).dt.dayofweek
features['sched_hour'] = pd.to_datetime(features['SKD_DTTM']).dt.hour
features['arrival_hour'] = pd.to_datetime(features['INB_ACT_DTTM']).dt.hour

#### Selecting features

Here we select the features that we will use as inputs to our model.

In [ ]:
# Print the features columns for easy review
features.columns

In [8]:
# Narrow the features list to the relevant features
features = features[['INB_FLIGHT', 'INB_ACT_DTTM', 'SEATS', 'INB_PAX', 'OUTB_PAX', 'MOGT',
       'AVL_GRND_TM', 'OPERAT_FLIGHT_NBR', 'RQRD_BOARD_MIN', 'SKD_DTTM',
       'ACT_DTTM', 'FIRST_BAG_OFF', 'FIRST_QRTL_BAG_OFF',
       'MEDIAN_BAG_OFF', 'THIRD_QRTL_BAG_OFF', 'LAST_BAG_OFF', 'FIRST_BAG_ON',
       'FIRST_QRTL_BAG_ON', 'MEDIAN_BAG_ON','THIRD_QRTL_BAG_ON', 'LAST_BAG_ON', 'FA_MIN_CREW',
       'FIRST_PAX_ON', 'FIRST_QRTL_PAX_ON', 'MEDIAN_PAX_ON',
       'THIRD_QRTL_PAX_ON', 'LAST_PAX_ON', 'TURN_TIME', 
        'AIRLINE_AA', 'FLEET_CD_319',
       'FLEET_CD_320', 'FLEET_CD_321', 'FLEET_CD_737', 'FLEET_CD_737',
       'FLEET_CD_757', 'FLEET_CD_E90', 'FLEET_CD_M80', 'SUB_FLEET_A319',
       'SUB_FLEET_A320', 'SUB_FLEET_A321', 'SUB_FLEET_A321-Transcon',
       'SUB_FLEET_B737-800', 'SUB_FLEET_B737-800 MAX', 'SUB_FLEET_B757-200',
       'SUB_FLEET_E190', 'SUB_FLEET_MD-80', 'SUB_FLEET_MD-83','INB_ACT_ORIG_ABI', 'INB_ACT_ORIG_ABQ', 'INB_ACT_ORIG_ACY',
       'INB_ACT_ORIG_AFW', 'INB_ACT_ORIG_ALB', 'INB_ACT_ORIG_AMA',
       'INB_ACT_ORIG_AMS', 'INB_ACT_ORIG_ANC', 'INB_ACT_ORIG_ANU',
       'INB_ACT_ORIG_ATL', 'INB_ACT_ORIG_AUA', 'INB_ACT_ORIG_AUS',
       'INB_ACT_ORIG_BAQ', 'INB_ACT_ORIG_BDA', 'INB_ACT_ORIG_BDL',
       'INB_ACT_ORIG_BGI', 'INB_ACT_ORIG_BGR', 'INB_ACT_ORIG_BHM',
       'INB_ACT_ORIG_BJX', 'INB_ACT_ORIG_BNA', 'INB_ACT_ORIG_BOG',
       'INB_ACT_ORIG_BOI', 'INB_ACT_ORIG_BON', 'INB_ACT_ORIG_BOS',
       'INB_ACT_ORIG_BSB', 'INB_ACT_ORIG_BTR', 'INB_ACT_ORIG_BTV',
       'INB_ACT_ORIG_BUF', 'INB_ACT_ORIG_BWI', 'INB_ACT_ORIG_BZE',
       'INB_ACT_ORIG_BZN', 'INB_ACT_ORIG_CAE', 'INB_ACT_ORIG_CAP',
       'INB_ACT_ORIG_CCS', 'INB_ACT_ORIG_CDG', 'INB_ACT_ORIG_CFG',
       'INB_ACT_ORIG_CHS', 'INB_ACT_ORIG_CLE', 'INB_ACT_ORIG_CLO',
       'INB_ACT_ORIG_CLT', 'INB_ACT_ORIG_CMH', 'INB_ACT_ORIG_CMW',
       'INB_ACT_ORIG_COS', 'INB_ACT_ORIG_CRP', 'INB_ACT_ORIG_CTG',
       'INB_ACT_ORIG_CUN', 'INB_ACT_ORIG_CUR', 'INB_ACT_ORIG_CVG',
       'INB_ACT_ORIG_CZM', 'INB_ACT_ORIG_DAY', 'INB_ACT_ORIG_DCA',
       'INB_ACT_ORIG_DEN', 'INB_ACT_ORIG_DFW', 'INB_ACT_ORIG_DSM',
       'INB_ACT_ORIG_DTW', 'INB_ACT_ORIG_DUB', 'INB_ACT_ORIG_EDI',
       'INB_ACT_ORIG_EGE', 'INB_ACT_ORIG_ELP', 'INB_ACT_ORIG_EUG',
       'INB_ACT_ORIG_EWR', 'INB_ACT_ORIG_FAT', 'INB_ACT_ORIG_FDF',
       'INB_ACT_ORIG_FLL', 'INB_ACT_ORIG_FOE', 'INB_ACT_ORIG_FSD',
       'INB_ACT_ORIG_GCM', 'INB_ACT_ORIG_GDL', 'INB_ACT_ORIG_GEG',
       'INB_ACT_ORIG_GGG', 'INB_ACT_ORIG_GJT', 'INB_ACT_ORIG_GLA',
       'INB_ACT_ORIG_GND', 'INB_ACT_ORIG_GRK', 'INB_ACT_ORIG_GRR',
       'INB_ACT_ORIG_GSO', 'INB_ACT_ORIG_GSP', 'INB_ACT_ORIG_GUA',
       'INB_ACT_ORIG_GUC', 'INB_ACT_ORIG_GYE', 'INB_ACT_ORIG_HAV',
       'INB_ACT_ORIG_HDN', 'INB_ACT_ORIG_HNL', 'INB_ACT_ORIG_HOG',
       'INB_ACT_ORIG_HOU', 'INB_ACT_ORIG_IAD', 'INB_ACT_ORIG_IAH',
       'INB_ACT_ORIG_ICT', 'INB_ACT_ORIG_ILM', 'INB_ACT_ORIG_IND',
       'INB_ACT_ORIG_JAC', 'INB_ACT_ORIG_JAX', 'INB_ACT_ORIG_JFK',
       'INB_ACT_ORIG_KEF', 'INB_ACT_ORIG_KIN', 'INB_ACT_ORIG_KOA',
       'INB_ACT_ORIG_LAS', 'INB_ACT_ORIG_LAX','INB_ACT_ORIG_LBB', 
        'INB_ACT_ORIG_LGA', 'INB_ACT_ORIG_LIH',
       'INB_ACT_ORIG_LIM', 'INB_ACT_ORIG_LIR', 'INB_ACT_ORIG_LIS',
       'INB_ACT_ORIG_LIT', 'INB_ACT_ORIG_MAD', 'INB_ACT_ORIG_MAN',
       'INB_ACT_ORIG_MAO', 'INB_ACT_ORIG_MAR', 'INB_ACT_ORIG_MBJ',
       'INB_ACT_ORIG_MCI', 'INB_ACT_ORIG_MCO', 'INB_ACT_ORIG_MDE',
       'INB_ACT_ORIG_MDT', 'INB_ACT_ORIG_MEM', 'INB_ACT_ORIG_MEX',
       'INB_ACT_ORIG_MFE', 'INB_ACT_ORIG_MGA', 'INB_ACT_ORIG_MHT',
       'INB_ACT_ORIG_MIA', 'INB_ACT_ORIG_MKE', 'INB_ACT_ORIG_MLB',
       'INB_ACT_ORIG_MLI', 'INB_ACT_ORIG_MSN', 'INB_ACT_ORIG_MSP',
       'INB_ACT_ORIG_MSY', 'INB_ACT_ORIG_MTJ', 'INB_ACT_ORIG_MYR',
       'INB_ACT_ORIG_MZT', 'INB_ACT_ORIG_NAS', 'INB_ACT_ORIG_OAK',
       'INB_ACT_ORIG_OGG', 'INB_ACT_ORIG_OKC', 'INB_ACT_ORIG_OMA',
       'INB_ACT_ORIG_ONT', 'INB_ACT_ORIG_ORD', 'INB_ACT_ORIG_ORF',
       'INB_ACT_ORIG_PAE', 'INB_ACT_ORIG_PAP', 'INB_ACT_ORIG_PBI',
       'INB_ACT_ORIG_PDX', 'INB_ACT_ORIG_PEI', 'INB_ACT_ORIG_PHL',
       'INB_ACT_ORIG_PHX', 'INB_ACT_ORIG_PIT', 'INB_ACT_ORIG_PLS',
       'INB_ACT_ORIG_PNS', 'INB_ACT_ORIG_POP', 'INB_ACT_ORIG_POS',
       'INB_ACT_ORIG_PSP', 'INB_ACT_ORIG_PTP', 'INB_ACT_ORIG_PTY',
       'INB_ACT_ORIG_PUB', 'INB_ACT_ORIG_PUJ', 'INB_ACT_ORIG_PVD',
       'INB_ACT_ORIG_PVR', 'INB_ACT_ORIG_PWM', 'INB_ACT_ORIG_RDU',
       'INB_ACT_ORIG_RFD', 'INB_ACT_ORIG_RIC', 'INB_ACT_ORIG_RNO',
       'INB_ACT_ORIG_ROC', 'INB_ACT_ORIG_ROW', 'INB_ACT_ORIG_RST',
       'INB_ACT_ORIG_RSW', 'INB_ACT_ORIG_RTB', 'INB_ACT_ORIG_SAL',
       'INB_ACT_ORIG_SAN', 'INB_ACT_ORIG_SAP', 'INB_ACT_ORIG_SAT',
       'INB_ACT_ORIG_SAV', 'INB_ACT_ORIG_SBA', 'INB_ACT_ORIG_SDF',
       'INB_ACT_ORIG_SDQ', 'INB_ACT_ORIG_SEA', 'INB_ACT_ORIG_SFO',
       'INB_ACT_ORIG_SHV', 'INB_ACT_ORIG_SJC', 'INB_ACT_ORIG_SJD',
       'INB_ACT_ORIG_SJO', 'INB_ACT_ORIG_SJU', 'INB_ACT_ORIG_SKB',
       'INB_ACT_ORIG_SLC', 'INB_ACT_ORIG_SMF', 'INB_ACT_ORIG_SNA',
       'INB_ACT_ORIG_SNN', 'INB_ACT_ORIG_SNU', 'INB_ACT_ORIG_SPS',
       'INB_ACT_ORIG_SRQ', 'INB_ACT_ORIG_STI', 'INB_ACT_ORIG_STL',
       'INB_ACT_ORIG_STT', 'INB_ACT_ORIG_STX', 'INB_ACT_ORIG_SXM',
       'INB_ACT_ORIG_SYR', 'INB_ACT_ORIG_TGU', 'INB_ACT_ORIG_TLH',
       'INB_ACT_ORIG_TPA','INB_ACT_ORIG_TUL', 'INB_ACT_ORIG_TUS', 
        'INB_ACT_ORIG_TYS',
       'INB_ACT_ORIG_UIO', 'INB_ACT_ORIG_UVF', 'INB_ACT_ORIG_VPS',
       'INB_ACT_ORIG_VRA', 'INB_ACT_ORIG_VVI', 'INB_ACT_ORIG_XNA',
       'INB_ACT_ORIG_YUL', 'INB_ACT_ORIG_YVR', 'INB_ACT_ORIG_YYZ',
       'INB_ACT_ORIG_ZIH', 'INB_ACT_ORIG_ZSA','STATION_TYPE_DOM SPOKE',
       'STATION_TYPE_HUB/GW', 'STATION_TYPE_INTL',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_ABQ', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_ALB',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_AMA', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_AMS',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_ANC', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_ANU',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_ATL', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_AUA',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_AUS', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_BAQ',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_BDA', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_BDL',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_BGI', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_BHX',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_BJX', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_BNA',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_BOG', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_BOI',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_BON', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_BOS',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_BSB', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_BTV',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_BUF', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_BWI',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_BZE', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_BZN',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_CAE', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_CAP',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_CCS', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_CDG',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_CFG', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_CHS',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_CLE', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_CLT',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_CMH', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_CMW',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_COS', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_CTG',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_CUN', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_CUR',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_CVG', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_CZM',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_DAY', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_DCA',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_DEN', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_DFW',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_DSM', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_DTW',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_DUB', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_EDI',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_EGE', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_ELP',
        'SCHD_LEG_DEP_AIRPRT_IATA_CD_EWR', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_FAT',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_FDF', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_FLL',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_FPO', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_GCM',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_GDL', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_GEG',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_GEO', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_GLA',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_GND', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_GRR',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_GSO', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_GSP',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_GUA', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_GUC',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_GYE', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_HAV',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_HDN', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_HNL',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_HOG', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_IAD',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_IAH', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_ICT',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_ILM', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_IND',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_JAC', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_JAX',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_JFK', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_KEF',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_KIN', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_KOA',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_LAS', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_LAX',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_LBB', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_LGA',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_LIH', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_LIM',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_LIR', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_LIS',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_LIT', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MAD',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MAN', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MAO',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MAR', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MBJ',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MCI', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MCO',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MDE', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MDT',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MEM', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MEX',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MFE', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MGA',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MHT', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MIA',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MKE', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MSP',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MSY', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MTJ',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_MYR', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_MZT',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_NAS', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_OAK',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_OGG', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_OKC',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_OMA', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_ONT',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_ORD', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_ORF',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_PAP', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_PBI',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_PDX', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_PEI',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_PHL', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_PHX',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_PIT', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_PLS',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_PNS', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_POP',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_POS', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_PSP',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_PTP', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_PTY',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_PUJ', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_PVD',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_PVR', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_PWM',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_RDU', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_RIC',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_RNO', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_ROC',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_RSW', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_RTB',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SAL', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SAN',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SAP', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SAT',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SAV', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SBA',
        'SCHD_LEG_DEP_AIRPRT_IATA_CD_SDF', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SDQ',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SEA', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SFO',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SJC', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SJD',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SJO', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SJU',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SKB', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SLC',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SMF', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SNA',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SNN', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SNU',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SRQ', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_STI',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_STL', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_STT',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_STX', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_SXM',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_SYR', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_TGU',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_TPA', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_TUL',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_TUS', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_UIO',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_UVF', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_VPS',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_VRA', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_VVI',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_XNA', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_YUL',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_YVR', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_YYZ',
       'SCHD_LEG_DEP_AIRPRT_IATA_CD_ZIH', 'SCHD_LEG_DEP_AIRPRT_IATA_CD_ZSA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ABQ', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ALB',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_AMA', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_AMS',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ANC', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ANU',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ATL', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_AUA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_AUS', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BAQ',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BDA', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BDL',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BGI', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BHX',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BJX', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BNA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BOG', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BOI',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BON', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BOS',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BSB', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BTV',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BUF', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BWI',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BZE', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_BZN',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CAE', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CAP',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CCS', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CDG',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CFG', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CHS',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CLE', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CLO',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CLT', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CMH',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CMW', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_COS',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CRP', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CTG',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CUN', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CUR',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CVG', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_CZM',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_DAY', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_DCA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_DEN', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_DFW',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_DSM', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_DTW',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_DUB', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_EDI',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_EGE', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ELP',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_EWR', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_FAT',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_FDF', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_FLL',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GCM', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GDL',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GEG', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GEO',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GLA', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GND', 
        'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GRR', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GSO',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GSP', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GUA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GUC', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_GYE',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_HAV', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_HDN',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_HNL', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_HOG',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_IAD', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_IAH',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ICT', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ILM',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_IND', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_JAC',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_JAX', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_JFK',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_KEF', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_KIN',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_KOA', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LAS',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LAX', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LBB',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LGA', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LIH',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LIM', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LIR',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LIS', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LIT',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_LPB', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MAD',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MAN', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MAO',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MAR', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MBJ',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MCI', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MCO',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MDE', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MDT',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MEM', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MEX',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MFE', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MGA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MHT', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MIA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MKE', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MLB',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MSP', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MSY',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MTJ', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MYR',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_MZT', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_NAS',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_OAK', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_OGG',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_OKC', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_OMA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ONT', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ORD',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ORF', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PAP',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PBI', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PDX',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PEI', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PHL',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PHX', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PIT',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PLS', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PNS',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_POP', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_POS',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PSP', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PTP',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PTY', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PUJ',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PVD', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PVR',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_PWM', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_RDU',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_RIC', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_RNO',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ROC', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_RSW',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_RTB', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SAL',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SAN', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SAP',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SAT', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SAV',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SBA', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SDF',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SDQ', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SEA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SFO', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SJC',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SJD', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SJO',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SJU', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SKB',
        'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SLC', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SMF',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SNA', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SNN',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SNU', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SRQ',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_STI', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_STL',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_STT', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_STX',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SXM', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_SYR',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_TGU', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_TPA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_TUL', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_TUS',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_UIO', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_UVF',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_VPS', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_VRA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_VVI', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_XNA',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_YUL', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_YVR',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_YYZ', 'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ZIH',
       'SCHD_LEG_ARVL_AIRPRT_IATA_CD_ZSA', 'PRIMARY_DELAY_REASON_ATC',
       'PRIMARY_DELAY_REASON_CATERING', 'PRIMARY_DELAY_REASON_CONNECTIONS',
       'PRIMARY_DELAY_REASON_DAMAGE', 'PRIMARY_DELAY_REASON_FLIGHT ATTENDANT',
       'PRIMARY_DELAY_REASON_FLIGHT CREW', 'PRIMARY_DELAY_REASON_FUELING',
       'PRIMARY_DELAY_REASON_IT',
       'PRIMARY_DELAY_REASON_LATE ARRIVING EQUIPMENT',
       'PRIMARY_DELAY_REASON_LOAD PLANNING',
       'PRIMARY_DELAY_REASON_LOADING BAGGAGE/CARGO',
       'PRIMARY_DELAY_REASON_MAINTENANCE',
       'PRIMARY_DELAY_REASON_MISCELLANEOUS', 'PRIMARY_DELAY_REASON_NO DELAY',
       'PRIMARY_DELAY_REASON_OPERATIONAL CODES',
       'PRIMARY_DELAY_REASON_PASSENGER SERVICE', 'PRIMARY_DELAY_REASON_PILOT',
       'PRIMARY_DELAY_REASON_SECURITY', 'PRIMARY_DELAY_REASON_SERVICING',
       'PRIMARY_DELAY_REASON_UNKNOWN', 'PRIMARY_DELAY_REASON_WEATHER',
       'inb_pct_full', 'outb_pct_full', 'sched_month', 'sched_day_of_week',
       'sched_hour', 'arrival_hour']]

features.head()

,INB_FLIGHT,INB_ACT_DTTM,SEATS,INB_PAX,OUTB_PAX,MOGT,AVL_GRND_TM,OPERAT_FLIGHT_NBR,RQRD_BOARD_MIN,SKD_DTTM,...,PRIMARY_DELAY_REASON_SECURITY,PRIMARY_DELAY_REASON_SERVICING,PRIMARY_DELAY_REASON_UNKNOWN,PRIMARY_DELAY_REASON_WEATHER,inb_pct_full,outb_pct_full,sched_month,sched_day_of_week,sched_hour,arrival_hour
0,62,1503587040,128,125,57,45,25,2700,30,1503588540,...,0,0,0,0,0.976562,0.445312,1,3,0,0
1,120,1524318120,128,93,110,50,48,121,45,1524321000,...,0,0,0,0,0.726562,0.859375,1,3,0,0
2,149,1545669180,128,113,97,45,27,1454,30,1545670800,...,0,0,0,0,0.882812,0.757812,1,3,0,0
3,165,1528729140,128,89,85,40,-38,189,30,1528726860,...,0,0,0,0,0.695312,0.664062,1,3,0,0
4,189,1504093140,128,76,88,45,41,1534,30,1504095600,...,0,0,0,0,0.593750,0.687500,1,3,0,0


#### Extract features and labels

Here we create an array of features and an array of labels. The features will be the input to our model and the labels will be the outcome we are measuring (TURN_TIME).

In [9]:
imp_features = features

labels = np.array(features['TURN_TIME'])

features = features.drop(['TURN_TIME'], axis=1)

feature_list = list(features.columns)

features = np.array(features)

#### Split the data into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = \
train_test_split(features, labels, test_size = 0.25)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

## Train Model

We train the model with Scikit-Learn's RandomForestRegressor.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100)

rf.fit(train_features, train_labels)

## Make Predictions on the Test Set

Predict the turn time for each flight in the test set and record the average error in seconds and percentage.

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = rf.predict(test_features)

# Calculate the Mean Absolute Error
mae = round(mean_absolute_error(test_labels, predictions)/60,2)
print('Mean Absolute Error:', mae, 'minutes.')

# Calculate the Mean Absolute Percent Error
errors = abs(predictions - test_labels)
mape = 100 * (errors / test_labels)

accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%')

## Final Results

Here we create a DataFrame of the testing set, complete with our prediction, the real-world outcome, and a column comparing the two.

In [ ]:
# Create a DataFrame using the testing set
testdf = pd.DataFrame(test_features, columns=feature_list)

# Add the prediction and actual outcome columns
testdf['prediction'] = predictions
testdf['actual'] = test_labels

# Add the prediction and actual under_MOGT columns
testdf['actual_under_MOGT'] = testdf['actual'] <= (testdf['MOGT'] * 60)
testdf['pred_under_MOGT'] = testdf['prediction'] <= (testdf['MOGT'] * 60)

# Add a column to show if our prediction was correct
testdf['correct'] = testdf['actual_under_MOGT'] == testdf['pred_under_MOGT']


testdf.head()

#### Accuracy Score

The following score shows our accuracy in guessing if the plane will be turned under or over MOGT.

In [ ]:
print('Accuracy Score: {:.2%}'.format(testdf.correct.sum() / len(testdf.correct)))

## Variable Importances

In [ ]:
# Feature importance to predicting the dependent variable
feature_imp = pd.Series(rf.feature_importances_,index=feature_list).sort_values(ascending=False)
feature_imp = feature_imp[:30]
df = pd.DataFrame(feature_imp)
features_list= df.index
features_list

#### Visualize the relative feature importances

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Creating a bar plot
plt.figure(figsize=(15,7))
sns.barplot(x=feature_imp, y=feature_imp.index)

# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

## Create Random Forest Model with Important Features

In [10]:
imp_features = imp_features[['TURN_TIME', 'MOGT', 
                                'AVL_GRND_TM', 'PRIMARY_DELAY_REASON_MAINTENANCE',
                                'PRIMARY_DELAY_REASON_WEATHER', 'PRIMARY_DELAY_REASON_ATC',
                                'outb_pct_full', 'PRIMARY_DELAY_REASON_NO DELAY',
                                'PRIMARY_DELAY_REASON_FLIGHT CREW',
                                'PRIMARY_DELAY_REASON_LATE ARRIVING EQUIPMENT', 'OUTB_PAX',
                                'OPERAT_FLIGHT_NBR', 'INB_FLIGHT', 'PRIMARY_DELAY_REASON_PILOT',
                                'INB_PAX', 'inb_pct_full', 'PRIMARY_DELAY_REASON_DAMAGE',
                                'PRIMARY_DELAY_REASON_FLIGHT ATTENDANT', 'STATION_TYPE_HUB/GW',
                                'LAST_BAG_ON', 'FIRST_PAX_ON', 'ACT_DTTM', 'FIRST_QRTL_PAX_ON',
                                'THIRD_QRTL_PAX_ON', 'FA_MIN_CREW', 'SKD_DTTM', 'INB_ACT_DTTM',
                                'MEDIAN_BAG_ON', 'FIRST_BAG_ON', 'MEDIAN_BAG_OFF',
                                'THIRD_QRTL_BAG_OFF']]
labels = np.array(imp_features['TURN_TIME'])
imp_features = imp_features.drop(['TURN_TIME'], axis=1)
imp_features_list=imp_features.columns
imp_features = np.array(imp_features)

### K Fold Cross Validation 

This section randomly splits the dataset into K folds. For each K fold, the model is built around K-1 folds of the dataset. The model is tested for effectiveness on the Kth fold.

In [11]:
# K fold cross validation
from sklearn.model_selection import RepeatedKFold
kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=None) 

for train_index, test_index in kf.split(imp_features):
      print("Train:", train_index, "Validation:",test_index)
      X_train, X_test = imp_features[train_index], imp_features[test_index] 
      y_train, y_test = labels[train_index], labels[test_index]

Train: [     0      1      2 ... 158057 158058 158059] Validation: [    12     20     24 ... 158033 158051 158056]
Train: [     0      2      3 ... 158057 158058 158059] Validation: [     1     10     19 ... 158042 158043 158049]
Train: [     0      1      2 ... 158057 158058 158059] Validation: [     7     15     23 ... 158022 158027 158047]
Train: [     0      1      2 ... 158057 158058 158059] Validation: [     6      8     16 ... 158018 158031 158032]
Train: [     0      1      2 ... 158056 158057 158059] Validation: [    13     18     22 ... 158044 158048 158058]
Train: [     0      1      2 ... 158056 158058 158059] Validation: [     3     14     35 ... 158040 158055 158057]
Train: [     0      1      3 ... 158057 158058 158059] Validation: [     2      4      9 ... 158039 158041 158046]
Train: [     1      2      3 ... 158057 158058 158059] Validation: [     0      5     11 ... 158038 158045 158053]
Train: [     0      1      2 ... 158056 158057 158058] Validation: [    21     2

Train: [     0      1      2 ... 158057 158058 158059] Validation: [     4      7     10 ... 158005 158012 158053]
Train: [     0      1      3 ... 158057 158058 158059] Validation: [     2     34     35 ... 158038 158039 158040]
Train: [     0      1      2 ... 158056 158058 158059] Validation: [    20     22     61 ... 158042 158043 158057]
Train: [     0      1      2 ... 158056 158057 158059] Validation: [     8     36     47 ... 158047 158054 158058]
Train: [     0      1      3 ... 158056 158058 158059] Validation: [     2     13     39 ... 158050 158051 158057]
Train: [     0      1      2 ... 158057 158058 158059] Validation: [     9     31     38 ... 158032 158037 158044]
Train: [     0      2      3 ... 158056 158057 158058] Validation: [     1      4     22 ... 158024 158040 158059]
Train: [     0      1      2 ... 158057 158058 158059] Validation: [    12     15     40 ... 158048 158049 158052]
Train: [     1      2      3 ... 158057 158058 158059] Validation: [     0     1

### Train RF Model

Here we're using a scikit learn randomf forest regressor and tuning the hyperparameters. A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2, 
                           min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                           max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                           bootstrap=True, oob_score=False, n_jobs=None, random_state=None, 
                           verbose=0, warm_start=False)
rf.fit(X_train, y_train)

### Make Prediction on Test Set

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = rf.predict(X_test)

# Calculate the Mean Absolute Error
mae = round(mean_absolute_error(y_test, predictions)/60,2)
print('Mean Absolute Error:', mae, 'minutes.')

# Calculate the Mean Absolute Percent Error
errors = abs(predictions - y_test)
mape = 100 * (errors / y_test)

accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%')

### Final Results

In [ ]:
# Create a DataFrame using the testing set
imptestdf = pd.DataFrame(X_test, columns=imp_features_list)

# Add the prediction and actual outcome columns
imptestdf['prediction'] = predictions
imptestdf['actual'] = y_test

# Add the prediction and actual under_MOGT columns
imptestdf['actual_under_MOGT'] = imptestdf['actual'] <= (imptestdf['MOGT'] * 60)
imptestdf['pred_under_MOGT'] = imptestdf['prediction'] <= (imptestdf['MOGT'] * 60)

# Add a column to show if our prediction was correct
imptestdf['correct'] = imptestdf['actual_under_MOGT'] == imptestdf['pred_under_MOGT']


imptestdf.head()

### Accuracy 

In [ ]:
print('Accuracy Score: {:.2%}'.format(imptestdf.correct.sum() / len(imptestdf.correct)))